In [1]:
!curl -s http://checkip.amazonaws.com

3.8.68.2


In [2]:
pip install mlflow


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import joblib
import boto3
import mlflow
import io # To work with in-memory binary streams
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier
import os

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


In [5]:
import mlflow


from mlflow.tracking import MlflowClient

# If connecting to a remote server
tracking_uri = "http://18.175.251.68:5000"
client = MlflowClient(tracking_uri=tracking_uri)

# List all experiments
experiments = client.search_experiments()
for exp in experiments:
    print(f"ID: {exp.experiment_id}, Name: {exp.name}")


ID: 4, Name: xgb_random_search_experiment
ID: 3, Name: MultimodelClassifications_MLFlow
ID: 0, Name: Default


In [9]:
# import pandas as pd

# bucket = 'mobile-price-pred-buckets'                # Replace with your bucket name
# file_key = 'train.csv'     # Replace with your object's key (file path within bucket)
# s3_uri = f's3://{bucket}/{file_key}'

# df = pd.read_csv(s3_uri)
# print(df.head())

# # df = pd.read_csv('s3://mobile-price-pred-buckets/train.csv')
# # print(df.head())


In [10]:
# Basic imports
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')
import warnings
warnings.filterwarnings('ignore')

# Load the training data
df = pd.read_csv('train.csv')

# Preview the dataset
print("📊 Shape of dataset:", df.shape)
df.head()
df.describe()

📊 Shape of dataset: (2000, 21)


,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
count,2000.000000,2000.0000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,...,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,1238.518500,0.4950,1.522250,0.509500,4.309500,0.521500,32.046500,0.501750,140.249000,4.520500,...,645.108000,1251.515500,2124.213000,12.306500,5.767000,11.011000,0.761500,0.503000,0.507000,1.500000
std,439.418206,0.5001,0.816004,0.500035,4.341444,0.499662,18.145715,0.288416,35.399655,2.287837,...,443.780811,432.199447,1084.732044,4.213245,4.356398,5.463955,0.426273,0.500116,0.500076,1.118314
min,501.000000,0.0000,0.500000,0.000000,0.000000,0.000000,2.000000,0.100000,80.000000,1.000000,...,0.000000,500.000000,256.000000,5.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000
25%,851.750000,0.0000,0.700000,0.000000,1.000000,0.000000,16.000000,0.200000,109.000000,3.000000,...,282.750000,874.750000,1207.500000,9.000000,2.000000,6.000000,1.000000,0.000000,0.000000,0.750000
50%,1226.000000,0.0000,1.500000,1.000000,3.000000,1.000000,32.000000,0.500000,141.000000,4.000000,...,564.000000,1247.000000,2146.500000,12.000000,5.000000,11.000000,1.000000,1.000000,1.000000,1.500000
75%,1615.250000,1.0000,2.200000,1.000000,7.000000,1.000000,48.000000,0.800000,170.000000,7.000000,...,947.250000,1633.000000,3064.500000,16.000000,9.000000,16.000000,1.000000,1.000000,1.000000,2.250000
max,1998.000000,1.0000,3.000000,1.000000,19.000000,1.000000,64.000000,1.000000,200.000000,8.000000,...,1960.000000,1998.000000,3998.000000,19.000000,18.000000,20.000000,1.000000,1.000000,1.000000,3.000000


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  2000 non-null   int64  
 1   blue           2000 non-null   int64  
 2   clock_speed    2000 non-null   float64
 3   dual_sim       2000 non-null   int64  
 4   fc             2000 non-null   int64  
 5   four_g         2000 non-null   int64  
 6   int_memory     2000 non-null   int64  
 7   m_dep          2000 non-null   float64
 8   mobile_wt      2000 non-null   int64  
 9   n_cores        2000 non-null   int64  
 10  pc             2000 non-null   int64  
 11  px_height      2000 non-null   int64  
 12  px_width       2000 non-null   int64  
 13  ram            2000 non-null   int64  
 14  sc_h           2000 non-null   int64  
 15  sc_w           2000 non-null   int64  
 16  talk_time      2000 non-null   int64  
 17  three_g        2000 non-null   int64  
 18  touch_sc

In [12]:
# Make a copy to avoid changing original
df_fe = df.copy()

# Avoid divide-by-zero errors
df_fe['sc_w'] = df_fe['sc_w'].replace(0, 0.1)
df_fe['mobile_wt'] = df_fe['mobile_wt'].replace(0, 0.1)

# New features
df_fe['screen_area'] = df_fe['sc_h'] * df_fe['sc_w']
df_fe['battery_per_weight'] = df_fe['battery_power'] / df_fe['mobile_wt']
df_fe['ppi'] = np.sqrt(df_fe['px_height']*2 + df_fe['px_width']*2) / df_fe['sc_w']

# Scale
from sklearn.preprocessing import StandardScaler
scale_cols = [
    'battery_power', 'ram', 'int_memory', 'mobile_wt',
    'screen_area', 'battery_per_weight', 'ppi'
]
scaler = StandardScaler()
df_fe[scale_cols] = scaler.fit_transform(df_fe[scale_cols])

# Check for any inf/NaN again (should be none)
print("Any infinities?", np.isinf(df_fe[scale_cols]).any().any())
print("Any NaNs?", df_fe[scale_cols].isna().any().any())

df_fe.head()

Any infinities? False
Any NaNs? False


,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range,screen_area,battery_per_weight,ppi
0,-0.902597,0,2.2,0,1,0,-1.380644,0.6,1.349249,2,...,9,7.0,19,0,0,1,1,-0.226205,-1.158914,-0.376091
1,-0.495139,1,0.5,1,0,1,1.155024,0.7,-0.120059,3,...,17,3.0,7,1,1,0,2,-0.382644,-0.454090,-0.260835
2,-1.537686,1,0.5,1,2,1,0.493546,0.9,0.134244,5,...,11,2.0,9,1,1,0,2,-0.760707,-1.297607,-0.183488
3,-1.419319,1,2.5,0,0,0,-1.215274,0.8,-0.261339,6,...,16,8.0,11,1,0,0,2,0.621177,-1.108662,-0.352384
4,1.325906,1,1.2,0,13,1,0.658915,0.6,0.021220,2,...,8,2.0,15,1,1,0,1,-0.838927,0.804356,-0.205742


In [13]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Split data
X = df_fe.drop('price_range', axis=1)
y = df_fe['price_range']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Store models and their names
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
}

# Train and evaluate
for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    
    print(f"🔍 {name} Results:")
    print("Accuracy:", accuracy_score(y_test, preds))
    print("Classification Report:")
    print(classification_report(y_test, preds))
    print("-" * 50)

🔍 Logistic Regression Results:
Accuracy: 0.8525
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.94      0.94       100
           1       0.81      0.73      0.77       100
           2       0.73      0.83      0.78       100
           3       0.94      0.91      0.92       100

    accuracy                           0.85       400
   macro avg       0.86      0.85      0.85       400
weighted avg       0.86      0.85      0.85       400

--------------------------------------------------
🔍 Random Forest Results:
Accuracy: 0.91
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.97      0.96       100
           1       0.87      0.89      0.88       100
           2       0.89      0.82      0.85       100
           3       0.92      0.96      0.94       100

    accuracy                           0.91       400
   macro avg       0.91      0.91      0.91       400


In [15]:
X_train.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,screen_area,battery_per_weight,ppi
469,1.380537,1,0.5,1,10,0,1.596010,0.3,-1.250297,1,...,-0.362588,9,5.0,6,0,0,0,-0.460864,2.271393,-0.357056
565,0.073938,1,0.5,0,12,0,-0.002563,0.1,-0.769946,7,...,-1.548431,15,14.0,14,0,0,1,1.690181,0.416386,-0.375074
396,-1.025518,0,2.5,1,4,1,1.375517,0.9,-1.391576,8,...,1.701110,12,8.0,6,1,1,1,0.204005,-0.185998,-0.373149
452,0.381239,0,0.5,1,2,1,0.107683,0.8,-1.193785,3,...,1.651316,9,8.0,2,1,0,1,-0.108875,1.137623,-0.358554
325,-0.725045,0,2.1,1,5,0,-0.388426,0.4,-0.713434,7,...,-1.234911,10,7.0,5,0,1,1,-0.134948,-0.339441,-0.366627


In [16]:
X_test.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,screen_area,battery_per_weight,ppi
501,1.216643,1,2.5,0,16,1,-0.002563,0.6,0.840642,6,...,1.329497,17,6.0,13,1,0,1,0.282224,0.225933,-0.337589
1924,1.692391,1,1.6,0,2,1,-1.105027,0.2,-1.702391,5,...,-0.276832,13,3.0,14,1,1,1,-0.539084,3.564447,-0.327270
1710,-1.571831,0,1.0,1,0,1,1.320394,0.4,0.162500,4,...,-1.440544,17,14.0,8,1,1,1,2.055207,-1.327706,-0.380582
547,-0.276613,1,0.6,1,0,0,-0.057686,0.2,0.162500,2,...,-0.111773,8,7.0,14,0,1,1,-0.317461,-0.420733,-0.345572
496,0.718132,1,0.5,1,11,0,-1.490890,1.0,-0.459131,3,...,1.393123,7,4.0,18,0,0,0,-0.682487,0.715309,-0.317197


In [17]:
from sklearn.model_selection import RandomizedSearchCV

# Define XGBoost model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)

# Define hyperparameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5, 6],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

# Initialize RandomizedSearchCV
xgb_random = RandomizedSearchCV(
    xgb_model, param_distributions=param_grid,
    n_iter=20, cv=3, scoring='accuracy', verbose=1, n_jobs=-1, random_state=42
)

# Fit on training data
xgb_random.fit(X_train, y_train)

# Best model
best_xgb = xgb_random.best_estimator_

# Evaluate on test set
xgb_preds = best_xgb.predict(X_test)
xgb_acc = accuracy_score(y_test, xgb_preds)

print("✅ Best XGBoost Accuracy:", round(xgb_acc, 4))
print("Classification Report:\n", classification_report(y_test, xgb_preds))

Fitting 3 folds for each of 20 candidates, totalling 60 fits


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms o

✅ Best XGBoost Accuracy: 0.9375
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.99      0.98       100
           1       0.94      0.92      0.93       100
           2       0.90      0.90      0.90       100
           3       0.95      0.94      0.94       100

    accuracy                           0.94       400
   macro avg       0.94      0.94      0.94       400
weighted avg       0.94      0.94      0.94       400



In [ ]:
import pickle

# Save the best model as a pickle file
with open("best_xgb_model.pkl", "wb") as f:
    pickle.dump(best_xgb, f)

print("✅ Model saved as best_xgb_model.pkl")


### Set up MLflow tracking & experiment

In [21]:
import os
print(os.getcwd())



/home/ec2-user/SageMaker/mobile_prediction_folder


In [22]:
os.listdir()


['.env.save',
 'best_xgb_model.pkl',
 'classification_report.json',
 'requirements.txt',
 '.ipynb_checkpoints',
 'mobile_price_prediction.ipynb',
 'train.csv',
 'mlruns',
 'Untitled.ipynb',
 'mobile_price_prediction.py',
 'feature_importances.png',
 '.env',
 'test.csv',
 'classification_report.txt',
 'testdata_mobile_price_predictions.ipynb',
 'Github_files',
 'confusion_matrix.png']

In [23]:
!pip install mlflow


import mlflow
from mlflow import sklearn as mlflow_sklearn

print(mlflow.__version__)


3.2.0


In [24]:
# # (A) Local tracking store (good for SageMaker/Jupyter)
# TRACKING_DIR = "/home/ec2-user/SageMaker/mobile_prediction_folder/mlruns"  # change if needed
# os.makedirs(TRACKING_DIR, exist_ok=True)
# mlflow.set_tracking_uri(f"file://{TRACKING_DIR}")
EC2_PUBLIC_IP = "18.175.251.68"
mlflow.set_tracking_uri(f"http://{EC2_PUBLIC_IP}:5000")

In [25]:
# (B) Name your experiment
EXPERIMENT_NAME = "xgb_random_search_experiment"
mlflow.set_experiment(EXPERIMENT_NAME)

<Experiment: artifact_location='s3://heartattackcsv/artifacts/4', creation_time=1755183973954, experiment_id='4', last_update_time=1755183973954, lifecycle_stage='active', name='xgb_random_search_experiment', tags={}>

### Enable autologging and run your search

In [ ]:
import mlflow.sklearn
mlflow.sklearn.autolog(log_input_examples=True, log_model_signatures=True)

from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier

# your existing objects: X_train, y_train, X_test, y_test, param_grid (as you defined)

xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)

xgb_random = RandomizedSearchCV(
    xgb_model,
    param_distributions=param_grid,
    n_iter=20,
    cv=3,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1,
    random_state=42
)

with mlflow.start_run(run_name="xgb_random_search") as run:
    xgb_random.fit(X_train, y_train)
    best_xgb = xgb_random.best_estimator_

    # Evaluate on test set and log
    from sklearn.metrics import accuracy_score, classification_report
    xgb_preds = best_xgb.predict(X_test)
    xgb_acc = accuracy_score(y_test, xgb_preds)
    mlflow.log_metric("test_accuracy", float(xgb_acc))
    print("✅ Best XGBoost Accuracy:", round(xgb_acc, 4))
    print("Classification Report:\n", classification_report(y_test, xgb_preds))


### Add rich artifacts (confusion matrix, report, feature importance)

In [26]:
import json, numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

with mlflow.start_run(run_name="xgb_best_eval", nested=True) as eval_run:
    # Confusion matrix
    cm = confusion_matrix(y_test, xgb_preds)
    disp = ConfusionMatrixDisplay(cm)
    plt.figure()
    disp.plot()
    plt.title("Confusion Matrix - Best XGB")
    plt.savefig("confusion_matrix.png", bbox_inches="tight")
    plt.close()
    mlflow.log_artifact("confusion_matrix.png")

    # Classification report as JSON and text
    report_dict = classification_report(y_test, xgb_preds, output_dict=True)
    with open("classification_report.json", "w") as f:
        json.dump(report_dict, f, indent=2)
    mlflow.log_artifact("classification_report.json")

    with open("classification_report.txt", "w") as f:
        f.write(classification_report(y_test, xgb_preds))
    mlflow.log_artifact("classification_report.txt")

    # Feature importance
    importances = getattr(best_xgb, "feature_importances_", None)
    if importances is not None:
        plt.figure()
        order = np.argsort(importances)[::-1][:30]  # top 30
        names = getattr(X_train, "columns", np.arange(len(importances)))
        names = np.array(names)[order]
        plt.bar(range(len(order)), importances[order])
        plt.xticks(range(len(order)), names, rotation=90)
        plt.title("Top Feature Importances (XGB)")
        plt.tight_layout()
        plt.savefig("feature_importances.png", bbox_inches="tight")
        plt.close()
        mlflow.log_artifact("feature_importances.png")


🏃 View run xgb_best_eval at: http://18.175.251.68:5000/#/experiments/4/runs/9a1f5f71cd46479d85cb9abb40f5e4b1
🧪 View experiment at: http://18.175.251.68:5000/#/experiments/4


<Figure size 640x480 with 0 Axes>

### Log the final model with signature & input example

In [27]:
from mlflow.models import infer_signature

# Infer signature (works whether X_* are pandas or numpy)
try:
    proba_or_pred = best_xgb.predict_proba(X_train[:5])
except Exception:
    proba_or_pred = best_xgb.predict(X_train[:5])
signature = infer_signature(X_train[:5], proba_or_pred)

with mlflow.start_run(run_name="xgb_model_logging", nested=True) as log_run:
    # Prefer xgboost flavor for XGBClassifier
    import mlflow.xgboost
    mlflow.xgboost.log_model(
        best_xgb,
        artifact_path="model",
        signature=signature,
        input_example=X_train[:5]
    )
    RUN_ID = log_run.info.run_id
    print("Model logged under run:", RUN_ID)


2025/08/18 13:20:31 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Model logged under run: 5bf55fa3467b4add8c30be202a795321
🏃 View run xgb_model_logging at: http://18.175.251.68:5000/#/experiments/4/runs/5bf55fa3467b4add8c30be202a795321
🧪 View experiment at: http://18.175.251.68:5000/#/experiments/4


### Register the model to the MLflow Model Registry

In [28]:
from mlflow.tracking import MlflowClient

MODEL_NAME = "xgb_classifier_randomsearch"

client = MlflowClient()
model_uri = f"runs:/{RUN_ID}/model"

# Create the registered model if it doesn't exist
registered = [m.name for m in client.search_registered_models()]
if MODEL_NAME not in registered:
    client.create_registered_model(MODEL_NAME)

# Create a new model version
mv = client.create_model_version(name=MODEL_NAME, source=model_uri, run_id=RUN_ID)
print("Registered Model Version:", mv.version)

# (Optional) transition stage: "Staging" or "Production"
client.transition_model_version_stage(
    name=MODEL_NAME,
    version=mv.version,
    stage="Staging",
    archive_existing_versions=False
)


2025/08/18 13:20:36 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: xgb_classifier_randomsearch, version 2


Registered Model Version: 2


<ModelVersion: aliases=[], creation_timestamp=1755523236850, current_stage='Staging', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1755523236887, metrics=None, model_id=None, name='xgb_classifier_randomsearch', params=None, run_id='5bf55fa3467b4add8c30be202a795321', run_link='', source='runs:/5bf55fa3467b4add8c30be202a795321/model', status='READY', status_message=None, tags={}, user_id='', version='2'>

In [18]:
import pickle

# Load the saved model from the file
with open("best_xgb_model.pkl", "rb") as f:
    loaded_model = pickle.load(f)

# Now you can use 'loaded_model' to make predictions
# predictions = loaded_model.predict(new_data)

In [19]:
predictions = loaded_model.predict(X_test)

In [20]:
predictions

array([3, 1, 0, 2, 3, 2, 0, 0, 1, 0, 3, 0, 0, 1, 3, 3, 2, 0, 1, 3, 1, 0,
       3, 2, 2, 1, 1, 0, 2, 1, 1, 2, 0, 2, 2, 3, 0, 3, 1, 0, 2, 0, 0, 2,
       3, 0, 3, 3, 2, 1, 0, 3, 0, 2, 0, 3, 3, 0, 2, 1, 1, 2, 3, 1, 0, 1,
       0, 1, 3, 3, 3, 2, 1, 1, 2, 0, 3, 3, 2, 3, 3, 2, 2, 0, 0, 1, 0, 0,
       0, 1, 0, 2, 2, 2, 0, 3, 0, 0, 2, 2, 3, 1, 1, 0, 3, 3, 1, 2, 1, 1,
       0, 2, 3, 1, 3, 2, 2, 2, 2, 2, 3, 0, 3, 0, 3, 3, 2, 0, 2, 3, 3, 0,
       0, 2, 3, 1, 1, 3, 2, 3, 3, 3, 2, 1, 3, 2, 1, 1, 0, 1, 3, 0, 0, 1,
       3, 3, 3, 2, 0, 2, 1, 0, 0, 3, 1, 1, 2, 1, 3, 0, 1, 1, 1, 3, 2, 1,
       1, 3, 0, 3, 2, 2, 2, 3, 0, 2, 0, 1, 1, 1, 0, 2, 2, 2, 3, 0, 2, 3,
       1, 1, 1, 2, 1, 3, 0, 0, 3, 1, 1, 1, 0, 3, 2, 0, 0, 3, 1, 2, 3, 3,
       0, 3, 2, 2, 2, 1, 1, 2, 0, 1, 0, 2, 0, 0, 0, 0, 0, 2, 2, 2, 2, 3,
       0, 1, 3, 2, 2, 3, 3, 2, 1, 0, 3, 3, 1, 3, 2, 2, 2, 2, 2, 2, 2, 1,
       0, 0, 3, 1, 2, 3, 2, 0, 0, 0, 1, 3, 2, 2, 3, 1, 3, 2, 2, 2, 1, 0,
       1, 1, 1, 3, 1, 0, 3, 1, 1, 1, 0, 1, 1, 1, 1,